In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\adkhare\\OneDrive - Informatica\\Desktop\\my_personal\\chicken_disease_MLOps\\chicken_disease_mlops'

Entity

In [14]:
from dataclasses import dataclass # dataclass decorator is used to create a class with attributes and methods
from pathlib import Path

@dataclass(frozen=True) # frozen=True makes the class immutable
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [15]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


Configuration

In [26]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        
        create_directories([config.root_dir,
                            Path(model_ckpt_dir),
                            Path(config.tensorboard_root_log_dir)])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )    
        
        return prepare_callbacks_config
    

In [27]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

component

In [28]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
        
    @property
    def create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,
                                          f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True)
        
    def get_tb_ckpt_callbacks(self):
        return [self.create_tb_callbacks, self.create_ckpt_callbacks]    
        

pipeline

In [29]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e    

[2023-07-03 17:08:43,646: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-03 17:08:43,649: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-03 17:08:43,651: INFO: common: created directory at: artifacts]
[2023-07-03 17:08:43,653: INFO: common: created directory at: artifacts/prepare_callbacks]
[2023-07-03 17:08:43,655: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-07-03 17:08:43,657: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
